In [6]:
import pandas as pd
import numpy as np
import datetime
import csv

In [7]:
fileName = r'C:\Users\wangjuha\Desktop\DMG Raw Data Jul-Dec 2019 clean - 2.csv'

In [8]:
fileNameClean = fileName[:-4]+' - clean.csv'

In [9]:
fileNameClean

'C:\\Users\\wangjuha\\Desktop\\DMG Raw Data Jul-Dec 2019 clean - 2 - clean.csv'

In [13]:
df = pd.read_csv(fileName, encoding="unicode_escape")

,Unnamed: 0,Timestamp,ArtStorageUnit SpaceRH 15_Min(%),ArtStorageUnit SpaceTemp 15_Min(°F),GalleryUnit SpaceRH15_Min(%),GalleryUnit SpaceTemp15_Min(°F)
0,0,1/1/2020 0:00,50.2,71.5,45.3,67.2
1,1,1/1/2020 0:15,49.2,71.6,44.5,67.1
2,2,1/1/2020 0:30,50.4,71.5,45.2,67.2
3,3,1/1/2020 0:45,50.5,71.5,45.3,67.2
4,4,1/1/2020 1:00,50.1,71.8,45.1,67.2
...,...,...,...,...,...,...
631,631,1/7/2020 13:45,50.4,71.5,45.3,67.3
632,632,1/7/2020 14:00,50.4,71.6,45.0,67.5
633,633,1/7/2020 14:15,46.5,71.3,44.8,67.8
634,634,1/7/2020 14:30,52.2,71.7,44.9,67.5


In [13]:
col_1='ArtStorageUnit SpaceRH 15_Min(%)'
#Name of the first column on the right of Timestamp; Could be changed if needed;Same for below
col_2='ArtStorageUnit SpaceTemp 15_Min(F)'
col_3='GalleryUnit SpaceRH15_Min(%)'
col_4='GalleryUnit SpaceTemp15_Min(F)'

In [14]:
def check_chunck(col):
    wrongdf = df[df[col] == 0]
    if len(wrongdf.index)>13:
        for i in range(len(wrongdf.index)-13):
            if (wrongdf.index[i]==wrongdf.index[i+1]-1==wrongdf.index[i+2]-2==wrongdf.index[i+3]-3==wrongdf.index[i+4]-4==
            wrongdf.index[i+5]-5==wrongdf.index[i+6]-6==wrongdf.index[i+7]-7==wrongdf.index[i+8]-8==wrongdf.index[i+9]-9==
            wrongdf.index[i+10]-10==wrongdf.index[i+11]-11==wrongdf.index[i+12]-12):
                print("There is a large chunck of errors on " + wrongdf.loc[wrongdf.index[i],"Timestamp"])

check_chunck(col_1)
check_chunck(col_2)
check_chunck(col_3)
check_chunck(col_4)




In [15]:
def modify(row_num,col_name):
    #replace the value of a cell with the average of its upper cell and its lower cell
    df.loc[row_num,col_name]=(df.loc[row_num+1,col_name]+df.loc[row_num-1,col_name])/2

In [16]:
len(df.loc[(df[col_1]<=20)|(df[col_1]>=75)])
#For testing purpose

0

In [17]:
def automodify():
#Check for errors in Relative Humidity and fix when incorrect(below 20% or above 75%)
    for i in range(len(df.loc[(df[col_1]<=20)|(df[col_1]>=75)])):

        modify(df.loc[(df[col_1]<=20)|(df[col_1]>=75)].index[0],col_1)
    for i in range(len(df.loc[(df[col_3]<=20)|(df[col_3]>=75)])):

        modify(df.loc[(df[col_3]<=20)|(df[col_3]>=75)].index[0],col_3)
    
#Check for errors in Temperature and fix when incorrect(below 40F(around 5 celsius) or above 120F(around 45 celsius))
    for i in range(len(df.loc[(df[col_2]<=40)|(df[col_2]>=120)])):

        modify(df.loc[(df[col_2]<=40)|(df[col_2]>=120)].index[0],col_2)
    for i in range(len(df.loc[(df[col_4]<=40)|(df[col_4]>=120)])):

        modify(df.loc[(df[col_4]<=40)|(df[col_4]>120)].index[0],col_4)
automodify()

In [20]:
time_list = df['Timestamp'].tolist()    
def check_time(df):
#Conduct verify_time function to all the timestamps
    #correct=True
    j=0
    k=1
    for i in range(len(time_list)-1):
        if(i>0):
            col_1_val=(df.loc[i+1,col_1]+df.loc[i-1,col_1])/2
            col_2_val=(df.loc[i+1,col_2]+df.loc[i-1,col_2])/2
            col_3_val=(df.loc[i+1,col_3]+df.loc[i-1,col_3])/2
            col_4_val=(df.loc[i+1,col_4]+df.loc[i-1,col_4])/2
        if (not verify_large_missing_time(i)):
            print('OMG')
            print(i)
            line = pd.DataFrame({"Timestamp":(datetime.datetime.strptime(time_list[i], '%m/%d/%Y %H:%M')+ datetime.timedelta(minutes=15)).strftime('%m/%d/%Y %H:%M')},index=[i])
            df=pd.concat([df.iloc[:i+j+k], line, df.iloc[i+j+k:]]).reset_index(drop=True)
            j=j+1
                
            
        elif (not verify_time(i)):
            print("There is a false time at "+time_list[i]+" to "+time_list[i+1])
            #Create a new line with plus 15 minutes and data of the average of the upper cell data and lower cell data
            line = pd.DataFrame({"Timestamp":(datetime.datetime.strptime(time_list[i], '%m/%d/%Y %H:%M')+ datetime.timedelta(minutes=15)).strftime('%m/%d/%Y %H:%M'),
                    col_1:col_1_val,col_2:col_2_val ,col_3: col_3_val ,col_4: col_4_val},index=[i])
            df=pd.concat([df.iloc[:i+j+k], line, df.iloc[i+j+k:]]).reset_index(drop=True)
            k=k+1
            #correct=False
#Report false if anything is wrong; will be used in while loops
    #df.to_csv(fileNameClean,sep=',')
    return df
def verify_time(i):
#Verify if the time under timestamp matches with its lower cell
    later=(datetime.datetime.strptime(time_list[i], '%m/%d/%Y %H:%M')+ datetime.timedelta(minutes=15))
    if (later==datetime.datetime.strptime(time_list[i+1], '%m/%d/%Y %H:%M')):
        return True
    else:
        return False
def verify_large_missing_time(i):
#Verify if the time under timestamp matches with its lower cell
    correct=False
    later=(datetime.datetime.strptime(time_list[i], '%m/%d/%Y %H:%M')+ datetime.timedelta(minutes=15))
    for j in range(12):
        if (later==datetime.datetime.strptime(time_list[i+1], '%m/%d/%Y %H:%M')):
            correct=True
        else:
            later=later + datetime.timedelta(minutes=15)
    return correct
#verify_large_missing_time(5245)
df1=check_time(df)
#'%m/%d/%Y %H:%M'
#'%Y-%m-%d %H:%M:%S'

There is a false time at 1/4/2020 16:30 to 1/4/2020 17:00


C:\Users\wangjuha\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



TypeError: strptime() argument 1 must be str, not float

In [19]:
for i in range(3):
    df1=check_time(df1)

NameError: name 'df1' is not defined

In [160]:
df1.to_csv(fileNameClean,sep=',')

In [ ]:
##############################SCRATCH##############################

In [297]:
print(wrong_list[0])

6/30/2019 18:15


In [298]:
later=(datetime.datetime.strptime(wrong_list[0], '%m/%d/%Y %H:%M')+ dt.timedelta(minutes=15))
before=(datetime.datetime.strptime(wrong_list[0], '%m/%d/%Y %H:%M')- dt.timedelta(minutes=15))

In [299]:
later_str = later.strftime('%m/%d/%Y %H:%M')
print(later_str)

06/30/2019 18:30


In [317]:
i=df.loc[8,'ArtStorageUnit SpaceRH 15_Min(%)']
print(i)

50.4


In [302]:
wrongdf.loc["ArtStorageUnit SpaceRH 15_Min(%)",wrong_list[0]] = df.loc["ArtStorageUnit SpaceRH 15_Min(%)",wrong_list[0]+ dt.timedelta(minutes=15)]

TypeError: can only concatenate str (not "datetime.timedelta") to str

In [327]:
wrongdf = df[df['ArtStorageUnit SpaceRH 15_Min(%)'] == 50.4]